In [1]:
import os
import pandas as pd
from model_config import Path
import plotly.graph_objects as go 
import numpy as np
from scipy import stats
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import RobustScaler
import pandas_ta as ta
from sklearn.preprocessing import LabelEncoder

- All Fiches

In [2]:
scaler = StandardScaler()
timeframe = 1
data_num = 2
folder_path = Path["raw"]

In [3]:
if type(scaler) == type(StandardScaler()):
    scaler_name = "Standart"
    
elif type(scaler) == type(MinMaxScaler()):
    scaler_name = "MinMax"

elif type(scaler) == type(RobustScaler()):
    scaler_name = "Robust"

In [4]:
# Define numerical features to scale
numerical_features = [
                    "Close",
                    'Open', 'High', 'Low', 'Average',  'Change', 'Volume', 'Volume Change',
                    'EMA', 'SMA',  'MACD', 'BB_upper', 'BB_middle', 'BB_lower'
                      ]

In [5]:
csv_files = [f for f in os.listdir(folder_path) if f.endswith('.csv')]
dataframes = []

In [6]:
# Loop through each CSV file in the folder
for csv_file in csv_files:
    # Construct full file path
    file_path = os.path.join(folder_path, csv_file)
    
    # Read the CSV file into a DataFrame
    df = pd.read_csv(file_path)

    df = df.iloc[1:].reset_index(drop=True)
    
    # Extract the Asset_ID from the file name (assuming file name is the Asset_ID)
    asset_id = os.path.splitext(csv_file)[0].split("_")[0]  # Removes the '.csv' extension
    
    # Add the Asset_ID column to the DataFrame
    df['Asset_ID'] = asset_id
    
    # Append the DataFrame to the list
    dataframes.append(df)

# Concatenate all DataFrames into one
combined_df = pd.concat(dataframes, ignore_index=True)

# Display the combined DataFrame
combined_df.head()

,Date,Open,High,Low,Average,Close,Change,Volume,Volume Change,Asset_ID
0,2024-04-20 04:00,0.267168,0.270103,0.266700,0.268401,0.270103,1.098558,22209.569331,-11.194502,0X0
1,2024-04-20 05:00,0.270103,0.270957,0.268974,0.269965,0.270877,0.286480,4846.973710,-78.176192,0X0
2,2024-04-20 06:00,0.270877,0.271038,0.269079,0.270058,0.270706,-0.063166,5100.507938,5.230774,0X0
3,2024-04-20 07:00,0.270706,0.271651,0.269340,0.270495,0.271651,0.349155,3040.306921,-40.392075,0X0
4,2024-04-20 08:00,0.271651,0.271870,0.271651,0.271760,0.271870,0.080565,420.500479,-86.169144,0X0


In [7]:
# Обработка пропущенных значений
combined_df.fillna(method='ffill', inplace=True)
combined_df.fillna(method='bfill', inplace=True)

C:\Users\McHomak\AppData\Local\Temp\ipykernel_22976\3639975876.py:2: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  combined_df.fillna(method='ffill', inplace=True)
C:\Users\McHomak\AppData\Local\Temp\ipykernel_22976\3639975876.py:3: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  combined_df.fillna(method='bfill', inplace=True)


In [8]:
combined_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 197915 entries, 0 to 197914
Data columns (total 10 columns):
 #   Column         Non-Null Count   Dtype  
---  ------         --------------   -----  
 0   Date           197915 non-null  object 
 1   Open           197915 non-null  float64
 2   High           197915 non-null  float64
 3   Low            197915 non-null  float64
 4   Average        197915 non-null  float64
 5   Close          197915 non-null  float64
 6   Change         197915 non-null  float64
 7   Volume         197915 non-null  float64
 8   Volume Change  197915 non-null  float64
 9   Asset_ID       197915 non-null  object 
dtypes: float64(8), object(2)
memory usage: 15.1+ MB


In [9]:
combined_df['Date'] = pd.to_datetime(combined_df['Date'])
combined_df.sort_values(by=['Asset_ID', 'Date'], inplace=True)
combined_df.reset_index(drop=True, inplace=True)

In [10]:
combined_df = combined_df.groupby('Asset_ID', group_keys=False)

In [11]:
def add_technical_indicators(group):
    group['EMA'] = ta.ema(group['Close'], length=14)
    group['SMA'] = ta.sma(group['Close'], length=14)
    group['RSI'] = ta.rsi(group['Close'], length=14)
    group['MACD'] = ta.macd(group['Close'])['MACD_12_26_9']
    bbands = ta.bbands(group['Close'], length=20)
    group['BB_upper'] = bbands['BBU_20_2.0']
    group['BB_middle'] = bbands['BBM_20_2.0']
    group['BB_lower'] = bbands['BBL_20_2.0']
    return group

combined_df = combined_df.apply(add_technical_indicators)
combined_df.fillna(method='bfill', inplace=True)

C:\Users\McHomak\AppData\Local\Temp\ipykernel_22976\1048405128.py:12: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  combined_df = combined_df.apply(add_technical_indicators)
C:\Users\McHomak\AppData\Local\Temp\ipykernel_22976\1048405128.py:13: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  combined_df.fillna(method='bfill', inplace=True)


In [12]:
combined_df['Hour'] = combined_df['Date'].dt.hour
combined_df['Day'] = combined_df['Date'].dt.dayofweek  # 0 = Monday
combined_df['Month'] = combined_df['Date'].dt.month

combined_df['Hour_sin'] = np.sin(2 * np.pi * combined_df['Hour']/24)
combined_df['Hour_cos'] = np.cos(2 * np.pi * combined_df['Hour']/24)

combined_df['Day_sin'] = np.sin(2 * np.pi * combined_df['Day']/7)
combined_df['Day_cos'] = np.cos(2 * np.pi * combined_df['Day']/7)

combined_df['Month_sin'] = np.sin(2 * np.pi * combined_df['Month']/12)
combined_df['Month_cos'] = np.cos(2 * np.pi * combined_df['Month']/12)

In [13]:
label_encoder = LabelEncoder()
combined_df['Asset_ID_encoded'] = label_encoder.fit_transform(combined_df['Asset_ID'])

In [14]:
combined_df = combined_df.groupby('Asset_ID_encoded').apply(lambda x: x.iloc[48:]).reset_index(drop= True)

C:\Users\McHomak\AppData\Local\Temp\ipykernel_22976\4205974642.py:1: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  combined_df = combined_df.groupby('Asset_ID_encoded').apply(lambda x: x.iloc[48:]).reset_index(drop= True)


In [15]:
encoded_labels = combined_df['Asset_ID_encoded'].unique()
label_mapping = {encoded: original for encoded, original in zip(encoded_labels, label_encoder.inverse_transform(encoded_labels))}
print(label_mapping)

{0: '0X0', 1: 'AAVE', 2: 'AMPL', 3: 'ANDY', 4: 'APU', 5: 'ATH', 6: 'BANANA', 7: 'BASEDAI', 8: 'BOBO', 9: 'DEAI', 10: 'DRAGONX', 11: 'EIGEN', 12: 'ENA', 13: 'ENS', 14: 'FET', 15: 'FIGHT', 16: 'FLOKI', 17: 'GME', 18: 'HOPPY', 19: 'ILV', 20: 'JESUS', 21: 'JOE', 22: 'KENDU', 23: 'LDO', 24: 'LINK', 25: 'MAGA', 26: 'MOG', 27: 'NEIRO', 28: 'NEURAL', 29: 'NPC', 30: 'ONDO', 31: 'PAAL', 32: 'PAXG', 33: 'PEIPEI', 34: 'PENDLE', 35: 'PEPE', 36: 'PEPECOIN', 37: 'PRIME', 38: 'SHFL', 39: 'SHIB', 40: 'SMURFCAT', 41: 'SUPER', 42: 'TITANX', 43: 'TRUMP', 44: 'UNI', 45: 'WLD', 46: 'WOJAK', 47: 'WOLF', 48: 'WQUIL', 49: 'WTAO', 50: 'ZYN'}


In [16]:
combined_df["Close_orig"] = combined_df["Close"]

In [17]:
def scale_data(df, scaler):
    if scaler in ["log"]:
        def scale_group(group):
            # Применяем логарифмическую трансформацию к числовым признакам
            group[numerical_features] = np.log(group[numerical_features] + 1e-6)  # Добавляем небольшое значение для избежания логарифма от 0
            return group
    else:
        def scale_group(group):
            group[numerical_features] = scaler.fit_transform(group[numerical_features])
            return group  

    df = df.groupby('Asset_ID', group_keys=False).apply(scale_group).reset_index(drop=True)
    return df

In [18]:
combined_df = scale_data(df = combined_df, scaler= scaler)

C:\Users\McHomak\AppData\Local\Temp\ipykernel_22976\2103416537.py:12: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df = df.groupby('Asset_ID', group_keys=False).apply(scale_group).reset_index(drop=True)


In [19]:
# Функция для замены нулей на среднее
def replace_zeros_with_mean(df, columns):
    for column in columns:
        df[column].replace(0, np.nan, inplace=True)
        df[column].interpolate(method='linear', inplace=True)
        df[column].fillna(method='bfill', inplace=True)
        df[column].fillna(method='ffill', inplace=True)
    return df

# Задаем столбцы, в которых нужно заменить нули
columns_to_replace = [ 
                      "Close_orig",   
                      "Close", 'Open', 'High', 'Low', 'Average',  'Change', 'Volume', 'Volume Change',
                      'EMA', 'SMA', 'RSI', 'MACD', 'BB_upper', 'BB_middle', 'BB_lower'
                      ]

In [20]:
# Замена нулевых значений на средние с интерполяцией для каждого Asset_ID
combined_df = combined_df.groupby('Asset_ID', group_keys=False).apply(lambda group: replace_zeros_with_mean(group, columns_to_replace)).reset_index(drop=True)

C:\Users\McHomak\AppData\Local\Temp\ipykernel_22976\4021808225.py:4: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df[column].replace(0, np.nan, inplace=True)
C:\Users\McHomak\AppData\Local\Temp\ipykernel_22976\4021808225.py:5: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For exam

In [21]:
zero_close_prices = combined_df[combined_df['Close'] == 0]
print(f"Number of zero 'Close' prices after scaling: {len(zero_close_prices)}")

Number of zero 'Close' prices after scaling: 0


In [22]:
combined_df.sort_values(by=['Asset_ID', 'Date'], inplace=True)
combined_df.drop(columns= ["Asset_ID", 'Date'], axis=1, inplace=True)

1) scaler
2) columns cnt
3) tokens cnt
4) timeframe

In [23]:
columns_cnt = combined_df.shape[1]
tokens_cnt = len(combined_df['Asset_ID_encoded'].unique())

In [24]:
combined_df.to_csv(Path["dataset"](data_num, scaler_name, columns_cnt, tokens_cnt, timeframe), index= False)
print(f"File saved to {Path['dataset'](data_num, scaler_name, columns_cnt, tokens_cnt, timeframe)}")

File saved to d:\PythonScripts\RL_for_Trading\data\processed\dataset_2D_Standart_26cols_51tkn_1t.csv


In [25]:
df = pd.read_csv(Path["dataset"](data_num, scaler_name, columns_cnt, tokens_cnt, timeframe))

In [26]:
df.columns

Index(['Open', 'High', 'Low', 'Average', 'Close', 'Change', 'Volume',
       'Volume Change', 'EMA', 'SMA', 'RSI', 'MACD', 'BB_upper', 'BB_middle',
       'BB_lower', 'Hour', 'Day', 'Month', 'Hour_sin', 'Hour_cos', 'Day_sin',
       'Day_cos', 'Month_sin', 'Month_cos', 'Asset_ID_encoded', 'Close_orig'],
      dtype='object')

In [27]:
# Функция для вычисления разницы цен для одного актива
def calculate_price_differences(asset_data):
    asset_data = asset_data.sort_values('Date')  # Сортируем по дате
    
    # Начальная и конечная цена
    start_price = asset_data['Close_orig'].iloc[0]
    end_price = asset_data['Close_orig'].iloc[-1]
    
    # Абсолютная и относительная разница между начальной и конечной ценой
    abs_diff_start_end = end_price - start_price
    rel_diff_start_end = (end_price - start_price) / start_price * 100
    
    # Максимальная и минимальная цена
    max_price = asset_data['Close_orig'].max()
    min_price = asset_data['Close_orig'].min()
    
    # Абсолютная и относительная разница между максимальной и минимальной ценой
    abs_diff_max_min = max_price - min_price
    rel_diff_max_min = (max_price - min_price) / min_price * 100
    
    return pd.Series({
        'start_price': start_price,
        'end_price': end_price,
        'abs_diff_start_end': abs_diff_start_end,
        'rel_diff_start_end': rel_diff_start_end,
        'max_price': max_price,
        'min_price': min_price,
        'abs_diff_max_min': abs_diff_max_min,
        'rel_diff_max_min': rel_diff_max_min
    })

In [28]:
# result = df.groupby('Asset_ID_encoded').apply(calculate_price_differences).reset_index()
# columns_to_round = ['abs_diff_start_end', 'rel_diff_start_end', 'abs_diff_max_min', 'rel_diff_max_min']
# result[columns_to_round] = result[columns_to_round].round(3)
# result.head()